In [4]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import time
import progressbar
from scipy import stats

In [5]:
filenames = ['../../data/bookcorssing_r05.csv',
             '../../data/bookcorssing_r10.csv',
             '../../data/audioscrobbler.csv',  
             '../../data/amazonbook.csv',
             '../../data/amazoncd.csv',  
             '../../data/amazonelectronic.csv',
             '../../data/amazonmovie.csv',
             '../../data/facebooklike_msg.csv',
             '../../data/algorithm_r05.csv',
             '../../data/artificialintelligence_r05.csv',
             '../../data/programminglanguage_r05.csv'
            ]
threshs = [625, 625, 85, 3578, 990, 278, 785, 100, 550, 725, 675]
blocks = [0,0,0,0,0,0,0,0,0,0,0]

In [7]:
def get_valid_data(filename,block):
    '''    first ,kick out invalid data -- rating time less than thresh   '''
    #allData=pd.read_csv('BX-Book-Ratings.csv')
    #print(allData.columns )
    allData=dict()
    pos = 0
    with open(filename,'r') as f:
            # filter the first line
        f.readline()  
        lines=f.readlines()
            # traverse and filter 
        for idx,line in enumerate(lines):
            if block == 1:
                dataInString=line.strip().split(';')  # list type
            else:
                dataInString=line.strip().split(',')  # list type
            for index,data in enumerate(dataInString):  # userName ISBN score
                dataInString[index]=data.replace('\"','')
            if not allData.get(dataInString[1]) :   # create the user's list if he does not exist
                allData[dataInString[1]]=[]
            allData[dataInString[1]].append(float(dataInString[2]))
    #        miu.append( float(dataInString[2]) )
    return allData
    '''calculate miu1 and miu2 using allData but predict only with valid users'''

In [8]:
def get_valid_user(allData, thresh):
    # %% cell1
    ''' pick out valid user '''
    #num=0
    miu=[]    # to store all data
    rateFreq=dict() # key is rate and value is its frequency  rateFreq[userName][score]=frequency
    for key in allData.keys():      # userName
        if len(allData[key])> thresh:
    #        num+=1
                # create the dict for each valid user
            if not rateFreq.get( key ):
                rateFreq[ key ]= dict()     
                # create the dict entry for each score
            for score in allData[key]:
                if not rateFreq[key].get(score):
                    rateFreq[ key ][ score ]=0
                rateFreq[ key ][ score ]+=1
                miu.append(score)
    return rateFreq, miu

In [9]:
def cal_para_multiple(rateFreq, miu, write_multiple):
    # %% cell2
    '''calculate common para'''
    miu1=np.mean(miu);  theta1=np.std(miu)
    # four direction for miu2 and theta2 
    # deltamiu=[0.5,-0.5]; theta=[2,1,-1,-2]
    cp = [(0,1), (0,-1), (1,-1), (-1,1)]

    write=open(write_multiple,'w');

    N=8*len(rateFreq)
    p = progressbar.ProgressBar(maxval=N)
    p.start() 
    progress=0

    userWeight=[None]*8   # list of 4 ele
    for i in range(2):
        for j in range(4):
            deltamiu = cp[j][0]
            theta = cp[j][1]

            miu2=miu1+deltamiu
            theta2=theta1+theta

            userWeight[i*4+j]=dict()
                # rateFreq[userName][score]=frequency
                # userScore[score]=frequency
            '''test'''
            print("************************************",file=write)
            print('i={0},j={1}'.format(i,j),file=write )
            sumMse_all = 0.0
            for user,userScore in rateFreq.items(): 
                    # cal MSE and choose the best weight
                bestWeight=0
                minMse=float('inf');
                '''对每个user 遍历一遍weight 把每个数据点都相加'''
                for weight in np.arange(0, 1.1 ,0.1): # arange exclude endPoint while linspace includes defaultly  
                    sumMse=0
                        # userScore[score]=frequency
                    '''对于每个user，他对每个score的mse要累计起来'''
                    '''这里需要check下data'''
                    for score,freq in userScore.items():    
                        sumMse+=(( weight*stats.norm(miu1, theta1).cdf(score)+ (1-weight)*stats.norm(miu2,theta2).cdf(score)- freq ) )**2
                    
                    '''test'''
                    print('user:{0},weight:{1},mse:{2}'.format(user,weight,sumMse),file=write )
                   
                    if sumMse < minMse:
                        minMse=sumMse
                        bestWeight=weight
                sumMse_all += minMse
                '''test'''
                print('user:{0},minMse:{1}'.format(user,minMse),file=write )
                userWeight[i*4+j][user]=bestWeight;   # change here   
                
                progress+=1
                p.update(progress)
            print(sumMse_all/len(rateFreq.keys()))
            print(sumMse_all/len(rateFreq.keys()),file=write )

    
    print(userWeight,file=write)
    write.close()
    p.finish()

In [10]:
def cal_para_single(rateFreq, miu, write_single):
    # %% cell2
    '''calculate common para'''
    miu1=np.mean(miu);  theta1=np.std(miu)
    # four direction for miu2 and theta2 
    # deltamiu=[0.5,-0.5]; theta=[2,1,-1,-2]
    cp = [(0,1), (0,-1), (1,-1), (-1,1)]

    write=open(write_single,'w');


    N=8*len(rateFreq)
    p = progressbar.ProgressBar(maxval=N)
    p.start() 
    progress=0

    userWeight=[None]*8   # list of 4 ele
    for i in range(2):
        for j in range(1):
            
            userWeight[i*4+j]=dict()
                # rateFreq[userName][score]=frequency
                # userScore[score]=frequency
            '''test'''
            print("************************************",file=write)
            print('i={0},j={1}'.format(i,j),file=write )
            sumMse_all = 0.0
            for user,userScore in rateFreq.items(): 
                    # cal MSE and choose the best weight
                bestWeight=0
                minMse=float('inf');
                '''对每个user 遍历一遍weight 把每个数据点都相加'''
                for weight in np.arange(0, 1.1 ,0.1): # arange exclude endPoint while linspace includes defaultly  
                    sumMse=0
                        # userScore[score]=frequency
                    '''对于每个user，他对每个score的mse要累计起来'''
                    '''这里需要check下data'''
                    for score,freq in userScore.items():    
                        sumMse+=((weight*stats.norm(miu1, theta1).cdf(score)- freq ))**2
                    
                    '''test'''
                    print('user:{0},weight:{1},mse:{2}'.format(user,weight,sumMse),file=write )
                   
                    if sumMse < minMse:
                        minMse=sumMse
                        bestWeight=weight
                sumMse_all += minMse
                '''test'''
                print('user:{0},minMse:{1}'.format(user,minMse),file=write )
                userWeight[i*4+j][user]=bestWeight;   # change here   
                
                progress+=1
                p.update(progress)
            print(sumMse_all/len(rateFreq.keys()),)
            print(sumMse_all/len(rateFreq.keys()),file=write)

    
    print(userWeight,file=write)
    write.close()
    p.finish()

# Data Regression item

## I=0 bookcorssing_r05

In [9]:
i = 0
c = filenames[i]
c = c.replace('ratings_', '')
s_out = c.replace('.csv','')+'_single.txt'
m_out = c.replace('.csv','')+'_multiple.txt'
print(s_out, m_out)
a_dict, a_miu = get_valid_user(get_valid_data(filenames[i], blocks[i]), threshs[i])
cal_para_single(a_dict, a_miu, s_out)
cal_para_multiple(a_dict, a_miu, m_out)

../../data/bookcorssing_r05_single.txt ../../data/bookcorssing_r05_multiple.txt


100% |########################################################################|


201980.62657483545
201980.62657483545
201980.62657483545


inf
inf
201724.43946938156
201980.62657483545


inf
inf
201724.43946938156


## I=1 bookcorssing_r10

In [10]:
i = 1
c = filenames[i]
c = c.replace('ratings_', '')
s_out = c.replace('.csv','')+'_single.txt'
m_out = c.replace('.csv','')+'_multiple.txt'
print(s_out, m_out)
a_dict, a_miu = get_valid_user(get_valid_data(filenames[i], blocks[i]), threshs[i])
cal_para_single(a_dict, a_miu, s_out)
cal_para_multiple(a_dict, a_miu, m_out)

../../data/bookcorssing_r10_single.txt ../../data/bookcorssing_r10_multiple.txt


100% |########################################################################|


103781.2579564965
103781.2579564965


103781.2579564965
103749.8142439248


103781.2579564965
103613.26789989739


103781.2579564965
103749.8142439248


103781.2579564965
103613.26789989739


## I=2 audioscrobbler

In [11]:
i = 3
c = filenames[i]
c = c.replace('ratings_', '')
s_out = c.replace('.csv','')+'_single.txt'
m_out = c.replace('.csv','')+'_multiple.txt'
print(s_out, m_out)
a_dict, a_miu = get_valid_user(get_valid_data(filenames[i], blocks[i]), threshs[i])
cal_para_single(a_dict, a_miu, s_out)
cal_para_multiple(a_dict, a_miu, m_out)

../../data/amazonbook_single.txt ../../data/amazonbook_multiple.txt


24333333.247492906


100% |########################################################################|


24333333.247492906


24333300.682893325


...

24331780.64942965


24333300.682893325


24332149.1463465


24333333.247492906


24331780.64942965


## I-3 amazonbook

In [12]:
i = 3
c = filenames[i]
c = c.replace('ratings_', '')
s_out = c.replace('.csv','')+'_single.txt'
m_out = c.replace('.csv','')+'_multiple.txt'
print(s_out, m_out)
a_dict, a_miu = get_valid_user(get_valid_data(filenames[i], blocks[i]), threshs[i])
cal_para_single(a_dict, a_miu, s_out)
cal_para_multiple(a_dict, a_miu, m_out)

../../data/amazonbook_single.txt ../../data/amazonbook_multiple.txt


24333333.247492906


100% |########################################################################|


24333333.247492906


24333300.682893325


...

24331780.64942965


24333300.682893325


24332149.1463465


24333333.247492906


24331780.64942965


## I=4 amazoncd

In [13]:
i = 4
c = filenames[i]
c = c.replace('ratings_', '')
s_out = c.replace('.csv','')+'_single.txt'
m_out = c.replace('.csv','')+'_multiple.txt'
print(s_out, m_out)
a_dict, a_miu = get_valid_user(get_valid_data(filenames[i], blocks[i]), threshs[i])
cal_para_single(a_dict, a_miu, s_out)
cal_para_multiple(a_dict, a_miu, m_out)

../../data/amazoncd_single.txt ../../data/amazoncd_multiple.txt


990614.7856054052


100% |########################################################################|


990614.7856054052


990608.056209525


...

990327.9363578076


990608.056209525


990216.7262144752


990614.7856054052


990327.9363578076


## I=5 amazonelectronic

In [14]:
i = 5
c = filenames[i]
c = c.replace('ratings_', '')
s_out = c.replace('.csv','')+'_single.txt'
m_out = c.replace('.csv','')+'_multiple.txt'
print(s_out, m_out)
a_dict, a_miu = get_valid_user(get_valid_data(filenames[i], blocks[i]), threshs[i])
cal_para_single(a_dict, a_miu, s_out)
cal_para_multiple(a_dict, a_miu, m_out)

../../data/amazonelectronic_single.txt ../../data/amazonelectronic_multiple.txt


681284.8268916276


681284.8268916276


681282.0395093893


681123.3894812764


...

681121.8159452211


681282.0395093893


681123.3894812764


681284.8268916276


681121.8159452211


## I=6 amazonmovie

In [15]:
i = 6
c = filenames[i]
c = c.replace('ratings_', '')
s_out = c.replace('.csv','')+'_single.txt'
m_out = c.replace('.csv','')+'_multiple.txt'
print(s_out, m_out)
a_dict, a_miu = get_valid_user(get_valid_data(filenames[i], blocks[i]), threshs[i])
cal_para_single(a_dict, a_miu, s_out)
cal_para_multiple(a_dict, a_miu, m_out)

../../data/amazonmovie_single.txt ../../data/amazonmovie_multiple.txt


2104762.0354091264


100% |########################################################################|


2104762.0354091264


2104748.8481407687


...

2104398.689085897


2104748.8481407687


2104428.311276775


2104762.0354091264


2104398.689085897


## I=7 facebooklike_msg

In [22]:
i = 7
c = filenames[i]
c = c.replace('ratings_', '')
s_out = c.replace('.csv','')+'_single.txt'
m_out = c.replace('.csv','')+'_multiple.txt'
print(s_out, m_out)
a_dict, a_miu = get_valid_user(get_valid_data(filenames[i], blocks[i]), threshs[i])
cal_para_single(a_dict, a_miu, s_out)
cal_para_multiple(a_dict, a_miu, m_out)

../../data/facebooklike_msg_single.txt ../../data/facebooklike_msg_multiple.txt


3301.0531053732852


100% |########################################################################|


3301.0531053732852


3299.245426648802


...

3291.721909368017


3299.245426648802


3301.0531053732852


3301.0531053732852


3291.721909368017


## I=8 algorithm

In [35]:
i = 8
c = filenames[i]
c = c.replace('ratings_', '')
s_out = c.replace('.csv','')+'_single.txt'
m_out = c.replace('.csv','')+'_multiple.txt'
print(s_out, m_out)
a_dict, a_miu = get_valid_user(get_valid_data(filenames[i], blocks[i]), threshs[i])
cal_para_single(a_dict, a_miu, s_out)
cal_para_multiple(a_dict, a_miu, m_out)

../../data/algorithm_r05_single.txt ../../data/algorithm_r05_multiple.txt


664406406.9755052


664406406.9755052


664405093.0493214


inf


...

664399224.1392608


664405093.0493214


inf


inf


664399224.1392608


## I=9 ai

In [36]:
i = 9
c = filenames[i]
c = c.replace('ratings_', '')
s_out = c.replace('.csv','')+'_single.txt'
m_out = c.replace('.csv','')+'_multiple.txt'
print(s_out, m_out)
a_dict, a_miu = get_valid_user(get_valid_data(filenames[i], blocks[i]), threshs[i])
cal_para_single(a_dict, a_miu, s_out)
cal_para_multiple(a_dict, a_miu, m_out)

../../data/artificialintelligence_r05_single.txt ../../data/artificialintelligence_r05_multiple.txt


1268289529.347677


1268289529.347677


1268287534.6594706


inf


...

1268278124.0185704


1268287534.6594706


inf


inf


1268278124.0185704


## I=10 pl

In [37]:
i = 10
c = filenames[i]
c = c.replace('ratings_', '')
s_out = c.replace('.csv','')+'_single.txt'
m_out = c.replace('.csv','')+'_multiple.txt'
print(s_out, m_out)
a_dict, a_miu = get_valid_user(get_valid_data(filenames[i], blocks[i]), threshs[i])
cal_para_single(a_dict, a_miu, s_out)
cal_para_multiple(a_dict, a_miu, m_out)

../../data/programminglanguage_r05_single.txt ../../data/programminglanguage_r05_multiple.txt


1671438277.431108


1671438277.431108


1671436493.6254094


inf


...

1671427832.758893


1671436493.6254094


inf


inf


1671427832.758893


In [11]:
c = "libimseti_f_m.csv"
s_out = c.replace('.csv','')+'_single.txt'
m_out = c.replace('.csv','')+'_multiple.txt'
filename = "../data/libimseti_f_m.csv"
block = 0
thresh = 100
print(s_out, m_out)
a_dict, a_miu = get_valid_user(get_valid_data(filename, block), thresh)
cal_para_single(a_dict, a_miu, s_out)
cal_para_multiple(a_dict, a_miu, m_out)

libimseti_f_m_single.txt libimseti_f_m_multiple.txt


NameError: name 'thresh' is not defined

In [12]:
c = "libimseti_m_f.csv"
s_out = c.replace('.csv','')+'_single.txt'
m_out = c.replace('.csv','')+'_multiple.txt'
filename = "../data/libimseti_m_f.csv"
block = 0
thresh = 100 
print(s_out, m_out)
a_dict, a_miu = get_valid_user(get_valid_data(filename, block), thresh)
cal_para_single(a_dict, a_miu, s_out)
cal_para_multiple(a_dict, a_miu, m_out)

libimseti_m_f_single.txt libimseti_m_f_multiple.txt


237613.93144211377


237613.93144211377


237601.9022537439


237599.7076313606


...

237561.16837302624


237601.9022537439


237599.7076313606


237612.50777446042


237561.16837302624
